# Queries for Blog 1

## Scenario 1: Worldwide gross of movies which has been shot in New Zealand, with minimum 7.5 rating.


In [ ]:
%%gremlin --store-to result

g.V().has('place', 'name', containing('New Zealand')).in().has('movie', 'rating', gt(7.5)).dedup().valueMap(['name', 'gross_worldwide', 'rating', 'studio','id'])

In [ ]:
import pandas as pd

df = pd.DataFrame(result)
for c in df:
    df[c] = df[c].apply(lambda x: x[-1])

df.sort_values("gross_worldwide", ascending=False).reset_index(drop=True).head(10)

## Scenario 2: Top 50 movies which belong to Action and Drama genre and has Oscar winning actors.

In [ ]:
%%gremlin --store result_action --silent

g.V().has('genre', 'name', 'Action').in().has('movie', 'rating', gt(8.5)).dedup().valueMap(['name', 'year', 'poster'])

In [ ]:
%%gremlin --store result_drama --silent

g.V().has('genre', 'name', 'Drama').in().has('movie', 'rating', gt(8.5)).dedup().valueMap(['name', 'year', 'poster'])

In [ ]:
%%gremlin --store result_actors --silent

g.V().has('person', 'oscar_winner', true).in().has('movie', 'rating', gt(8.5)).dedup().valueMap(['name', 'year', 'poster'])

In [ ]:
import pandas as pd
from IPython.display import Image, HTML

df_action = pd.DataFrame(result_action)
df_drama = pd.DataFrame(result_drama)
df_actors = pd.DataFrame(result_actors)

for df in [df_action, df_drama, df_actors]:
    for c in df:
        df[c] = df[c].apply(lambda x: x[-1])

In [ ]:
df = pd.merge(df_drama, df_action, on=["name", "year", "poster"])
df = pd.merge(df, df_actors, on=["name","year", "poster"])

df.poster = df.poster.apply(lambda x: f'<img src="{x}" style="max-height:50px"/>')

HTML(df.to_html(escape=False))

## Scenario 3: Movies which has common keyword "tattoo" and "assassin".

In [ ]:
%%gremlin --store result


g.V().has('keyword', 'name', 'assassin').in().has('keyword','name','tattoo').dedup().limit(10).valueMap(['name', 'year', 'poster'])

In [ ]:
import pandas as pd
from IPython.display import Image, HTML

df = pd.DataFrame(result)
for c in df:
    df[c] = df[c].apply(lambda x: x[-1])

df.poster = df.poster.apply(lambda x: f'<img src="{x}" style="max-height:30px"/>')

df.sort_values("year", ascending=False).reset_index(drop=True)

HTML(df.to_html(escape=False))

## Scenario 4: Movies that connect two actors Leonardo Dicaprio and Tom Hanks.

In [ ]:
%%gremlin --store result 

g.V().has('person', 'name', containing('Leonardo')).in().hasLabel('movie').out().hasLabel('person').has('name', 'Tom Hanks').path().by(valueMap('name', 'poster'))

In [ ]:
import pandas as pd
from IPython.display import Image, HTML

df = pd.DataFrame(result[0].objects)
for c in df:
    df[c] = df[c].apply(lambda x: x[-1])

df.poster = df.poster.apply(lambda x: f'<img src="{x}" style="max-height:30px"/>')

HTML(df.to_html(escape=False))